In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# import os
# os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async" # this seems to be highly important for totally utilizing your GPU's memory, but it also breaks the profiler's memory breakdown
# note that GradientTape needs several times the memory needed to compute the fidelity of a single circuit

import numpy as np
import qutip as qt
from QOGS.optimizer.tf_adam_optimizer import AdamOptimizer
from QOGS.gate_sets import PI_GRAPE
import matplotlib.pyplot as plt

In [ ]:
q_dim = 4
c_dim = 20
DAC_time_resolution = 2 # in ns
fock = 4

# In GHz = cycles / ns
anharm = .200
kerr = 1e-5
chi = 3e-3
drive = D = 2 * np.pi * 1e-3

a = qt.tensor(qt.destroy(c_dim), qt.qeye(q_dim)) # cavity
b = qt.tensor(qt.qeye(c_dim), qt.basis(q_dim, 0) * qt.basis(q_dim, 2).dag()) # qubit
ad = a.dag()
bd = b.dag()
H0 = (anharm/2) * bd * bd * b * b
H0 += (kerr/2) * ad * ad * a * a
H0 += (chi) * ad * a * bd * b
H0 *= 2*np.pi
Hcs = [D*(b + bd), 1j*D*(b - bd)]

init_states = [
    qt.tensor(qt.basis(c_dim, 0), qt.basis(q_dim, 0)),
    qt.tensor(qt.basis(c_dim, 2), qt.basis(q_dim, 0)),
    qt.tensor(qt.basis(c_dim, 4), qt.basis(q_dim, 0)),
]

final_states = [
    qt.tensor(qt.basis(c_dim, 0), qt.basis(q_dim, 2)),
    qt.tensor(-qt.basis(c_dim, 2), qt.basis(q_dim, 2)),
    qt.tensor(qt.basis(c_dim, 4), qt.basis(q_dim, 2))
]

jump_op = qt.tensor(qt.qeye(c_dim), qt.basis(q_dim, 2).proj())

In [ ]:
synth_params = {
    'N_blocks': 300, # note that the length of the pulse is this times the DAC_time_resolution
    'N_multistart' : 4, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 3000, #number of epochs before termination
    'epoch_size' : 5, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 1.02, #0.995, #terminal fidelitiy
    'dfid_stop' : 1e-13, #stop if dfid between two epochs is smaller than this number
    'initial_states' : init_states, #qubit tensor oscillator, start in |g> |0>
    'target_states' : final_states, #end in |e> |target>.
    'name' : 'PI GRAPE SNAP', #name for printing and saving
    'use_phase' : True,
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}

# We initialize the gateset here
gate_set_params = {
    'H_static' : H0,
    'H_control' : Hcs,
    'DAC_delta_t' : DAC_time_resolution,
    'inplace' : False, # true uses less memory, but is slower. Just use false
    'scale' : 1.0, # range of DAC amplitudes for initial random waves
    'bandwidth' : 0.1,
    'jump_ops' : jump_op,
    'jump_weights' : 1 / 25 / 300, # I need to check if this is delta_t * gamma_phi or delta_t * gamma_phi / number of blocks
    'gatesynthargs': synth_params
}



PI_GRAPE_gate_set = PI_GRAPE(**gate_set_params)

In [ ]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(PI_GRAPE_gate_set)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
PI_GRAPE_gate_set.best_fidelity()

In [ ]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
# Note that you will not want to use the performance profiler while using 'inplace' mode. You will run out of memory
opt.optimize()#logdir='logs')

In [ ]:
PI_GRAPE_gate_set.print_info()

In [ ]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
# Launch TensorBoard and navigate to the Profile tab to view performance profile
%tensorboard --logdir=logs